<a href="https://colab.research.google.com/github/byu-cs-452/byu-cs-452-class-content/blob/main/sparksql/02%20-%20SQLIntro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setup Spark SQL
# Note if running locally you need the JVM https://www.oracle.com/java/technologies/downloads/
# Consider running in https://colab.research.google.com/
%pip install pyspark

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession.builder.appName("SQLIntro").getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('WARN')


In [2]:
# Download Guteburg Book of Mormon and Bible
!curl -L https://ia601205.us.archive.org/18/items/thebookofmormon00017gut/mormon13.txt > bookOfMormon.txt
!curl -L https://www.gutenberg.org/cache/epub/10/pg10.txt > bible.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 87 1551k   87 1357k    0     0  1161k      0  0:00:01  0:00:01 --:--:-- 1162k
100 1551k  100 1551k    0     0  1291k      0  0:00:01  0:00:01 --:--:-- 1292k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 16 4351k   16  723k    0     0   791k      0  0:00:05 --:--:--  0:00:05  792k
 98 4351k   98 4277k    0     0  2233k      0  0:00:01  0:00:01 --:--:-- 2235k
100 4351k  100 4351k    0     0  2252k      0  0:00:01  0:00:01 --:--:-- 2253k


In [6]:
def putBookIntoWordsTable(bookFileTxt, tableName):
    import re
    lines = sc.textFile(bookFileTxt)
    words = lines.flatMap(lambda line: [i for i in re.split('[^a-zA-Z]', line.lower()) if i])
    asRows = words.map(lambda w: Row(word=w))
    asDF = asRows.toDF()
    asDF.createOrReplaceTempView(tableName)

putBookIntoWordsTable("bookOfMormon.txt", "bookOfMormon")
putBookIntoWordsTable("bible.txt", "bible")

In [7]:
bookOfMormonOnlyWords = spark.sql("""
                
                with uniqueToBookOfMormon as (
                    select * from bookOfMormon except select * from bible
                )
                select word, count(*) as counts from 
                bookOfMormon
                where word in (select word from uniqueToBookOfMormon)
                group by word
                order by counts desc
                
                """)

bookOfMormonOnlyWords.rdd.map(lambda row: (row.word, row.counts)).take(100)


[('nephi', 2812),
 ('alma', 2295),
 ('mosiah', 898),
 ('lamanites', 698),
 ('helaman', 595),
 ('moroni', 390),
 ('nephites', 372),
 ('mormon', 303),
 ('has', 298),
 ('chapter', 234),
 ('zarahemla', 148),
 ('towards', 101),
 ('forever', 100),
 ('show', 96),
 ('wo', 88),
 ('coriantumr', 76),
 ('amulek', 73),
 ('amalickiah', 64),
 ('limhi', 56),
 ('marvelous', 55),
 ('laman', 54),
 ('labor', 53),
 ('awful', 47),
 ('lamoni', 47),
 ('numerous', 45),
 ('teancum', 43),
 ('plan', 42),
 ('provisions', 37),
 ('abinadi', 37),
 ('pahoran', 37),
 ('omni', 34),
 ('zoramites', 33),
 ('shown', 32),
 ('commencement', 32),
 ('gadianton', 32),
 ('shiz', 31),
 ('guards', 30),
 ('happiness', 30),
 ('zeezrom', 28),
 ('murdered', 28),
 ('amlicites', 27),
 ('partake', 27),
 ('sacred', 27),
 ('showed', 27),
 ('ammonihah', 26),
 ('seashore', 26),
 ('shule', 26),
 ('jershon', 26),
 ('descendant', 25),
 ('morianton', 25),
 ('nephihah', 24),
 ('akish', 24),
 ('ammoron', 24),
 ('bloodshed', 22),
 ('manti', 22),
 ('

In [8]:
bibleOnlyWords = spark.sql("""
                
                with uniqueToBible as (
                    select * from bible except select * from bookOfMormon
                )
                select word, count(*) as counts from 
                bible
                where word in (select word from uniqueToBible)
                group by word
                order by counts desc
                
                """)

bibleOnlyWords.rdd.map(lambda row: (row.word, row.counts)).take(100)


[('levites', 265),
 ('shew', 228),
 ('cubits', 213),
 ('oil', 202),
 ('paul', 190),
 ('border', 158),
 ('honour', 145),
 ('joab', 145),
 ('shewed', 135),
 ('neighbour', 135),
 ('beside', 132),
 ('incense', 129),
 ('hezekiah', 128),
 ('court', 122),
 ('jonathan', 121),
 ('sister', 116),
 ('suburbs', 115),
 ('absalom', 108),
 ('bullock', 107),
 ('jeroboam', 104),
 ('esau', 100),
 ('plague', 98),
 ('ram', 98),
 ('shekels', 96),
 ('ahab', 94),
 ('threescore', 93),
 ('canaan', 91),
 ('labour', 89),
 ('pillars', 89),
 ('pharisees', 87),
 ('jehoshaphat', 85),
 ('daniel', 85),
 ('rams', 77),
 ('passover', 76),
 ('simon', 76),
 ('selah', 75),
 ('reuben', 74),
 ('eleazar', 74),
 ('table', 73),
 ('amorites', 73),
 ('hebron', 73),
 ('shouldest', 73),
 ('dan', 72),
 ('favour', 70),
 ('worshipped', 70),
 ('abode', 69),
 ('acts', 68),
 ('scribes', 68),
 ('horns', 67),
 ('bethel', 66),
 ('abimelech', 66),
 ('afterward', 66),
 ('chambers', 66),
 ('cherubims', 65),
 ('jericho', 64),
 ('shechem', 64),
 (